In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json

# read and merge 3 dfs

In [2]:
df1 = pd.read_csv('data/KR/krchall.csv',low_memory=False)
print(df1.shape)

(9844, 1552)


In [3]:
df2 = pd.read_csv('data/KR/krgm.csv',low_memory=False)
print(df2.shape)

(9191, 1552)


In [4]:
df3 = pd.read_csv('data/KR/krmaster.csv',low_memory=False)
print(df3.shape)

(12787, 1552)


In [5]:
df = pd.concat([df1,df2,df3])
print(df.shape)

(31822, 1552)


# Drop columns of single distinct value or >25% null

In [6]:
# keep only classic games

df = df[df['gameMode'] == "CLASSIC"]

In [7]:
# drop if one distinct value

for col in df.columns:
    if len(df[col].unique())==1:
        df = df.drop(col,axis=1)

In [8]:
# drop if null values are more than 25% of column

for col in df.columns:
    if (np.mean(df[col]!=df[col]) > .25):
        df = df.drop(col,axis=1)

In [9]:
print(df.shape)

(27480, 1107)


In [10]:
for col in df.columns:
    print(col + ": " + str(df[col].dtype))

gameId: int64
gameCreation: int64
gameDuration: int64
queueId: int64
gameVersion: object
teams_0_win: object
teams_0_firstBlood: bool
teams_0_firstTower: bool
teams_0_firstInhibitor: bool
teams_0_firstBaron: bool
teams_0_firstDragon: bool
teams_0_firstRiftHerald: bool
teams_0_towerKills: int64
teams_0_inhibitorKills: int64
teams_0_baronKills: int64
teams_0_dragonKills: int64
teams_0_riftHeraldKills: int64
teams_0_bans_0_championId: float64
teams_0_bans_0_pickTurn: float64
teams_0_bans_1_championId: float64
teams_0_bans_1_pickTurn: float64
teams_0_bans_2_championId: float64
teams_0_bans_2_pickTurn: float64
teams_0_bans_3_championId: float64
teams_0_bans_3_pickTurn: float64
teams_0_bans_4_championId: float64
teams_0_bans_4_pickTurn: float64
teams_1_win: object
teams_1_firstBlood: bool
teams_1_firstTower: bool
teams_1_firstInhibitor: bool
teams_1_firstBaron: bool
teams_1_firstDragon: bool
teams_1_firstRiftHerald: bool
teams_1_towerKills: int64
teams_1_inhibitorKills: int64
teams_1_baronKi

In [11]:
# fill after to allow int64 reconversion
df = df.fillna(0)

In [12]:
# drop if null values are more than 25% of column

for col in df.columns:
    if (df[col].dtypes == np.float64) and (sum(df[col]%1) == 0):
        df[col] = df[col].astype(np.int64)
    if (df[col].dtypes == bool):
        df[col] = df[col].astype(int)

In [13]:
df[['teams_0_win','teams_1_win']] = df[['teams_0_win','teams_1_win']].replace(['Win','Fail'],[1,0])

In [14]:
df

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,participantIdentities_8_player_currentAccountId,participantIdentities_8_player_matchHistoryUri,participantIdentities_8_player_profileIcon,participantIdentities_9_player_platformId,participantIdentities_9_player_accountId,participantIdentities_9_player_summonerName,participantIdentities_9_player_summonerId,participantIdentities_9_player_currentAccountId,participantIdentities_9_player_matchHistoryUri,participantIdentities_9_player_profileIcon
1,4536860675,1595927443677,2106,420,10.15.328.8759,1,0,1,1,0,...,3EadD5wyUGPgC25R6q0olzvFGfFGurFBJbDLy8aLUzSXag...,/v1/stats/player_history/KR/2327536718818880,23,KR,P_XR23fgb2oFIPRD6v19Fk3fjOzi5WOjHqIJTL3xnxfUAZI,DRX Gwangjalim,Okn79TBglp3BVJeBhMxTuVILx9erzJknQss2hNY9-MwbVGE,P_XR23fgb2oFIPRD6v19Fk3fjOzi5WOjHqIJTL3xnxfUAZI,/v1/stats/player_history/KR/210302380,4655
2,4536991748,1595932292463,1992,420,10.15.328.8759,1,1,1,1,1,...,WDYTPKK-VlB-ipcooLD6w1Y38VIfEv-Y741KbFVyknVKAI0,/v1/stats/player_history/KR/201931991,3522,KR,SEICSwdgqtI0DXi0FlGhLzXq0Z1lhKqIMX8c3PjB5TMz9oA,PESK,McYBCGI0qiIXb5fnScxcrdlwsimoa9aSypwTMAjRAruJTYE,SEICSwdgqtI0DXi0FlGhLzXq0Z1lhKqIMX8c3PjB5TMz9oA,/v1/stats/player_history/KR/200451279,4412
3,4478861319,1593442472883,1899,420,10.13.326.4870,1,1,1,1,1,...,Q4bcjGfFILpBav8RKwBU4nKqG7n4xfYxD0lzLO2mtAo7Xz...,/v1/stats/player_history/KR/2415880217612256,4535,KR,9FeKEso1d_PKy58RQa9uNrmPk7LAu-0FoaCRDOhWD1IPU6...,월드에디션,vLIG75wYX2NhM-ADq0vq6CuCFm-OIDpEHkhhYQmRJN_bdU4,9FeKEso1d_PKy58RQa9uNrmPk7LAu-0FoaCRDOhWD1IPU6...,/v1/stats/player_history/KR/2383287628164800,4025
5,4539023370,1596010510872,1314,420,10.15.330.344,0,0,0,0,0,...,nB7I2bN20JT4QL_auvDLA2NO17TznKVDzF3XRODZCeh9mY0,/v1/stats/player_history/KR/202971829,4568,KR,5_2v1m9BNviEj-AUo4bK1gi_uvxXZG3mBfRKlXcy8jOP_N...,hy2rwt,67YAKPcAoX-IM1DTZ2GJ4P7I4XlaLlPkE0ABjqql51nY23E,5_2v1m9BNviEj-AUo4bK1gi_uvxXZG3mBfRKlXcy8jOP_N...,/v1/stats/player_history/KR/1952335488027776,4535
6,4519952395,1595250773220,1039,420,10.14.327.8505,0,0,0,0,0,...,JuuNbVi8VgGhVzzMIbnCwHBwToZTk_BZ_FlGuyRR0Pehz7...,/v1/stats/player_history/KR/2372728168869344,29,KR,0tkTLNNVmQw4SfCZuUVJz9E_GyVi3heHRCcMXSjRXLEmmJ...,BIank Space,kLwVimt68waRiGM5K4JLX8PFVIcqzKi1zMRO0Qcw6TmrlH0,0tkTLNNVmQw4SfCZuUVJz9E_GyVi3heHRCcMXSjRXLEmmJ...,/v1/stats/player_history/KR/2372723298544736,4081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12782,4532076534,1595741977137,1790,420,10.15.328.8759,1,0,1,0,1,...,yTi1UEZ222DOJPwdtMkwnMEYRaMvH0gpNyRA0GL0WyR2MY...,/v1/stats/player_history/KR/2174430465683008,3182,KR,0mppFP0q_Ru2oWaEY6YEQFcrbkpryeBrfkFtugGzak0Me2...,clides,nH79bedewxpmmZ8JxG8p-VvMm7oX77ShLvGDwaXitfn8q6c,0mppFP0q_Ru2oWaEY6YEQFcrbkpryeBrfkFtugGzak0Me2...,/v1/stats/player_history/KR/2082398691208960,4589
12783,4504190967,1594539242916,2097,420,10.14.327.8505,0,0,1,0,0,...,hc3f_yzhbUphPHjvJkkm9AzRwPeJpzOIAkPf_K86eI0,/v1/stats/player_history/KR/691205,787,KR,OfKZDAWnzhfOpEGHeDp40Aj92_q2AuX71M12DBEgGmli84...,nice puppy,oXK7XT0H1nmmbSye-2bm9j9ZaIM94suly3LKaFHNbBCg1Sg,OfKZDAWnzhfOpEGHeDp40Aj92_q2AuX71M12DBEgGmli84...,/v1/stats/player_history/KR/2254726809095232,7
12784,4480630781,1593528551572,1416,420,10.13.326.4870,1,1,1,1,0,...,VF7VnUR2oXBxgVC4dHN0ItVic4dXUVcOxgDOwcki93tCQAs,/v1/stats/player_history/KR/207936842,4492,KR,qGFQ9JbCJi_NFPyhNpzadJrJ1esOMi4AOgAYXRvWgMV0,인선2,ENbGuTv8tEnuQMbEZdZh-0FPRm7DcIQNZ7P2bhG6o1dTPQ,qGFQ9JbCJi_NFPyhNpzadJrJ1esOMi4AOgAYXRvWgMV0,/v1/stats/player_history/KR/1157287,4568
12785,4546985982,1596300658657,1659,420,10.15.330.344,1,0,1,1,1,...,NVVBcyK5ntiTgKtYUGOj9i2aSzgZejWYPaYMqyL-SRXmKK4,/v1/stats/player_history/KR/201478650,4654,KR,cgCQcxJo_eNG-pJHvLb2fL12eL7E5rJ3iO2ZEboUsZ6Z8n...,정글차이노인정,lhRXYKteWpTHX1Zqcn8iSfzx_XxLqWllkcetVGdV6sNtlGI,cgCQcxJo_eNG-pJHvLb2fL12eL7E5rJ3iO2ZEboUsZ6Z8n...,/v1/stats/player_history/KR/2313134785769216,4568


In [15]:
#df.to_csv('data/clean.csv', index=False)